### This file converts selected CSV files from the MIMIC IV database into SQL databases
#### Authors: Zack Goldblum, Kevin Ramirez Chavez, Josh Miller 

In [10]:
import os
import pandas as pd
import sqlite3
from bmes import tempdir
from csv_to_sql import csv_to_sql
from sql_to_df import sql_to_df

# CSV file directory

csv_dir = os.path.join(tempdir(), "final_project_csv")
if not os.path.exists(csv_dir):
    os.mkdir(csv_dir)

# SQL database directory

sql_dir = os.path.join(tempdir(), "final_project_sql")
if not os.path.exists(sql_dir):
    os.mkdir(sql_dir)

print(f"The final project CSV directory is located at: {csv_dir}")

def create_sql_path(filename):
    """
    This function creates a SQL filepath

    Arguments
    ---------
        filename (str): name of the SQL database to create

    Returns
    -------
        None
    """

    return os.path.join(sql_dir, filename + ".sqlite")

The final project CSV directory is located at: C:/Users/zackg/AppData/Local/Temp/bmes\final_project_csv


### Extract the following CSV files from the MIMIC IV database to the final project CSV directory (above)
[MIMIC IV Dataset](https://physionet.org/content/mimiciv/2.1/)
* hosp/admissions.csv
* hosp/d_icd_diagnoses.csv
* hosp/diagnoses_icd.csv
* hosp/patients.csv
* hosp/pharmacy.csv
* icu/icustays.csv

### Create SQL databases with the corresponding columns

In [11]:
to_get_dict = {"patients": ["subject_id", "gender", "anchor_age"], 
               "pharmacy": ["subject_id", "medication"], 
               "admissions": ["subject_id", "deathtime"], 
               "diagnoses_icd": ["subject_id", "icd_code"], 
               "icustays": ["subject_id", "first_careunit"], 
               "d_icd_diagnoses": ["icd_code", "long_title"]}

for name, cols in to_get_dict.items():
    csv_to_sql(csv_dir, sql_dir, name, cols)

### Load all SQL databases into a Pandas DataFrame

In [12]:
# This table is created to obtain our specified variables from the patients table
pat = sql_to_df(create_sql_path("patients"), " SELECT subject_id, gender, anchor_age FROM patients")
print("Done patients")

# This table is created to obtain our specified variables from the pharmacy table
pharma = sql_to_df(create_sql_path("pharmacy"), " SELECT subject_id, medication FROM pharmacy")
# combine multiple subject_id rows into into one row with list of medications
pharma = pharma.groupby(['subject_id'])['medication'].apply(lambda x: ','.join(x.astype(str))).reset_index()
print("Done pharmacy")

# This table is created to obtain our specified variables from the admissions table
admi = sql_to_df(create_sql_path("admissions"), " SELECT subject_id, deathtime FROM admissions")
admi = admi.groupby(['subject_id'])['deathtime'].apply(lambda x: ','.join(x.astype(str))).reset_index()
print("Done admissions")

# This table is created to obtain our specified variables from the icustays table
stays = sql_to_df(create_sql_path("icustays"), " SELECT subject_id, first_careunit FROM icustays")
# combine multiple subject_id rows into into one row with list of stays
stays = stays.groupby(['subject_id'])['first_careunit'].apply(lambda x: ','.join(x.astype(str))).reset_index()
print("Done ICU stays")

# This table is created to obtain our specified variables from the diagnoses_icu table
diag = sql_to_df(create_sql_path("diagnoses_icd"), " SELECT subject_id, icd_code FROM diagnoses_icd")
# combine multiple subject_id rows into into one row with list of icd diagnoses
diag = diag.groupby(['subject_id'])['icd_code'].apply(lambda x: ','.join(x.astype(str))).reset_index()
print("Done diagnoses")

# Concatenating along columns
patient_data = pd.concat ([pat, pharma, admi, stays, diag], axis=1)
patient_data = patient_data.loc[:, ~patient_data.columns.duplicated()]

Done patients
Done pharmacy
Done admissions
Done ICU stays
Done diagnoses


### Store the PandasDataFrame with all relevant patient information into the patient_data SQL database

In [13]:
patient_data_filepath = create_sql_path("patient_data")

if not os.path.exists(patient_data_filepath):
    print("Creating patient_data.sqlite ...")
    db = sqlite3.connect(patient_data_filepath)
    
    db.execute("""CREATE TABLE   IF NOT EXISTS   patient_data (
        row_idx INTEGER PRIMARY KEY AUTOINCREMENT, 
        subject_id INTEGER, 
        gender VARCHAR(1), 
        anchor_age INTEGER, 
        medication TEXT, 
        deathtime TEXT, 
        first_careunit TEXT, 
        icd_code TEXT)
        """)
    
    for row in patient_data.itertuples():
        db.execute(""" INSERT INTO patient_data (subject_id, gender, anchor_age, medication, deathtime, first_careunit, icd_code) 
                VALUES (?, ?, ?, ?, ?, ?, ?) """, 
                (row.subject_id, row.gender, row.anchor_age, row.medication, row.deathtime, row.first_careunit, row.icd_code, )
                )
    
    db.commit()
    db.close()

    print("Done")

Creating patient_data.sqlite ...
Done


### Create a SQL database with unique pharmacy rows

In [14]:
from create_unique_pharm_sql import create_unique_pharm_sql

create_unique_pharm_sql()

Creating C:/Users/zackg/AppData/Local/Temp/bmes\final_project_sql\pharmacy_unique.sqlite
